In [5]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def get_unique_asignaturas(profesores_input):
    """Obtiene todas las asignaturas únicas y sus vacantes."""
    asignaturas = {}
    
    for profesor in profesores_input:
        for asignatura in profesor['Asignaturas']:
            codigo = asignatura['CodigoAsignatura']
            # Si la asignatura ya existe, verificamos que las vacantes sean las mismas
            if codigo in asignaturas:
                if asignaturas[codigo]['vacantes'] != asignatura['Vacantes']:
                    print(f"Advertencia: Asignatura {codigo} tiene diferentes números de vacantes")
                continue
            # Si es nueva, la agregamos
            asignaturas[codigo] = {
                'codigo': codigo,
                'nombre': asignatura['Nombre'],
                'vacantes': asignatura['Vacantes'],
                'campus': asignatura['Campus']
            }
    
    return asignaturas

def analyze_room_suitability(asignaturas, salas_input):
    """Analiza para cada asignatura cuántas salas son adecuadas."""
    resultados = []
    
    for codigo, info_asignatura in asignaturas.items():
        salas_adecuadas = 0
        salas_inadecuadas = 0
        
        # Para cada sala, verificar si es adecuada
        for sala in salas_input:

            if sala['Capacidad'] >= info_asignatura['vacantes']:
                salas_adecuadas += 1
            else:
                salas_inadecuadas += 1
        
        resultados.append({
            'Codigo_Asignatura': codigo,
            'Nombre_Asignatura': info_asignatura['nombre'],
            'Vacantes': info_asignatura['vacantes'],
            'Salas_Adecuadas': salas_adecuadas,
            'Salas_Inadecuadas': salas_inadecuadas
        })
    
    return resultados

def create_summary_table(resultados):
    """Crea una tabla resumen de las salas adecuadas por asignatura."""
    df_summary = pd.DataFrame(resultados)
    
    # Ordenar por número de salas adecuadas (ascendente) para identificar problemas potenciales
    df_summary = df_summary.sort_values('Salas_Adecuadas')
    
    # Guardar tabla en CSV
    df_summary.to_csv('resumen_re.csv', index=False)
    
    # Mostrar tabla formateada
    print("\nResumen de Salas Adecuadas por Asignatura:")
    print(df_summary.to_string(index=False))
    
    return df_summary

def create_adequacy_visualization(df_summary):
    """Crea visualizaciones para el análisis de adecuación."""
    plt.figure(figsize=(15, 6))
    
    # Ordenar por ratio de adecuación
    df_summary['Ratio_Adecuacion'] = (df_summary['Salas_Adecuadas'] / 
                                     (df_summary['Salas_Adecuadas'] + df_summary['Salas_Inadecuadas']))
    
    # Crear gráfico de dispersión
    plt.scatter(df_summary['Vacantes'], df_summary['Salas_Adecuadas'],
               alpha=0.6, c='#3CB371')
    plt.title('Relación entre Vacantes y Salas Adecuadas')
    plt.xlabel('Número de Vacantes')
    plt.ylabel('Salas Adecuadas')
    
    plt.tight_layout()
    plt.savefig('analisis_re.png')
    plt.close()

def save_results(df_summary):
    """Guarda los resultados en un archivo JSON."""
    try:
        # Calcular estadísticas globales
        total_asignaturas = len(df_summary)
        asignaturas_sin_salas = len(df_summary[df_summary['Salas_Adecuadas'] == 0])
        
        results = {
            'asignatura_stats': df_summary.to_dict(orient='records'),
            'global_stats': {
                'total_asignaturas': total_asignaturas,
                'asignaturas_sin_salas_adecuadas': asignaturas_sin_salas,
                'porcentaje_asignaturas_problematicas': (asignaturas_sin_salas / total_asignaturas * 100)
            }
        }
        
        with open('metricas_re.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos de entrada
    profesores_input = load_json_file('../../agent_input/InputOfProfesores.json')
    salas_input = load_json_file('../../agent_input/InputOfSala.json')
    
    if profesores_input is None or salas_input is None:
        return
    
    # 2. Obtener asignaturas únicas
    asignaturas = get_unique_asignaturas(profesores_input)
    
    # 3. Analizar adecuación de salas
    resultados = analyze_room_suitability(asignaturas, salas_input)
    
    # 4. Crear tabla resumen
    df_summary = create_summary_table(resultados)
    if df_summary is None:
        return
    
    # 5. Crear visualizaciones
    create_adequacy_visualization(df_summary)
    
    # 6. Guardar resultados
    if save_results(df_summary):
        print("\nAnálisis de RE completado exitosamente")
        
        # Identificar asignaturas problemáticas
        asignaturas_sin_salas = df_summary[df_summary['Salas_Adecuadas'] == 0]
        if not asignaturas_sin_salas.empty:
            print("\n¡ADVERTENCIA! Las siguientes asignaturas no tienen salas adecuadas:")
            print(asignaturas_sin_salas[['Codigo_Asignatura', 'Nombre_Asignatura', 'Vacantes', 'Campus']].to_string(index=False))
    else:
        print("Error al completar el análisis de RE")

if __name__ == "__main__":
    main()


Resumen de Salas Adecuadas por Asignatura:
Codigo_Asignatura         Nombre_Asignatura  Vacantes  Salas_Adecuadas  Salas_Inadecuadas
        (FIA14-A) INTRODUCCIÓN A LA INGENIE        67                5                 68
        (FIA14-B) INTRODUCCIÓN A LA INGENIE        61                6                 67
        (FIA14-C) INTRODUCCIÓN A LA INGENIE        63                6                 67
        (FIA41-B)             TERMODINÁMICA        53                7                 66
        (FIA22-C)            ÁLGEBRA LINEAL        47                8                 65
        (FIA11-A)       CÁLCULO DIFERENCIAL        47                8                 65
        (ARQ25-A)             MATEMATICAS I        46                8                 65
        (ARQ23-A)       HISTORIA Y TEORIA I        48                8                 65
        (PRI61-A)      OPERACIONES UNITARIA        46                8                 65
        (FIA22-A)            ÁLGEBRA LINEAL        49   